In [ ]:
# Import Dependencies.

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime
import time 
from scipy.stats import linregress

In [ ]:
# Create a set of random latitude and longitude combinations.

latitudes = np.random.uniform(low = -90, high = 90, size = 1500)
longitudes = np.random.uniform (low = -180, high = 180, size = 1500)
latitudes_longitudes = zip(latitudes,longitudes)
latitudes_longitudes

In [ ]:
# Unpack the longitudes and latitudes and put them in a list. 

coordinates = list(latitudes_longitudes)
len(coordinates)

In [ ]:
# Nearest cities for the above coordinates.

# Create a empty list that will hold all the cities name nearest to the above coordinates.
cities = []
# Identify the nearest city for each latitude and longitutde combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    # if city is unique then add to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

In [ ]:
# Basic for the OpenWeatherMap with weather_api_key added to the URL.

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=" + weather_api_key

In [ ]:
city_url = url + "&q=" + city
city_url

In [ ]:
# Create an empty list to hold weather data for cities.

city_data = []

# Print the begining of logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1

set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_description = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        try:
            city_rain = city_weather["rain"]["3h"]
        except:
            city_rain = 0
        try:
            city_snow = city_weather["snow"]["3h"]
        except:
            city_snow = 0
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_description,
                          "Rain inches (last 3 hrs)": city_rain,
                          "Snow inches (last 3 hrs)": city_snow})
    
    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass 
    
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
# Tally number of cities on the city_data.

len(city_data)

In [ ]:
# Create the DataFrame from the array of dictionaries.

city_data_df = pd.DataFrame(city_data)
city_data_df['Rain inches (last 3 hrs)'] = city_data_df['Rain inches (last 3 hrs)'] / 25.4
city_data_df['Snow inches (last 3 hrs)'] = city_data_df['Snow inches (last 3 hrs)'] / 25.4
city_data_df.head(10)

In [ ]:
# Create the output csv file.

output_data_file = "weather_data/WeatherPy_challenge.csv"
# Export the city_data into the csv file.
city_data_df.to_csv(output_data_file, index_label = "City_ID")

In [ ]:
# How many cities have recorded rainfall?

city_rain = city_data_df.loc[city_data_df['Rain inches (last 3 hrs)'] > 0]
city_rain['Rain inches (last 3 hrs)'].count()

In [ ]:
# How many cities have recorded snow?

city_snow = city_data_df.loc[city_data_df['Snow inches (last 3 hrs)'] > 0]
city_snow['Snow inches (last 3 hrs)'].count()

In [ ]:
# Format the DataFrame.

city_data_df['Rain inches (last 3 hrs)'] = city_data_df['Rain inches (last 3 hrs)'].map('{:.3f}'.format)
city_data_df['Snow inches (last 3 hrs)'] = city_data_df['Snow inches (last 3 hrs)'].map('{:.0f}'.format)

city_data_df.head(10)